In [ ]:
import cv2
import urllib.request
import requests
import numpy as np
import socket

# Load the face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://10.0.0.15:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

cv2.namedWindow('Video Stream', cv2.WINDOW_NORMAL)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500) 
        #buffer += stream.read(10000)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        #print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            cv2.imshow('Video Stream', frame)

            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
            
            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Define the ESP32's IP address and port number
                ip = "10.0.0.15"
                port = 81
    
                print("I see you!")

                # Define the integer to send
                message = 32

                # Create a socket object
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

                # Connect to the ESP32
                client_socket.connect((ip, port))

                # Send the integer as a string
                client_socket.send(str(message).encode())
                
                # Send number 32 to ESP32 over WiFi
                requests.get('http://10.0.0.15/?data=32')
                response = requests.get(url + '/?data=32')
                print(response.text)
                print("Data send!")

                # Close the socket connection
                client_socket.close()
                
                
                
            # Display the output
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()


In [ ]:
import cv2
import urllib.request
import numpy as np

# Define the ESP32 Wrover's IP address and port number
url = 'http://10.0.0.15:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            # Display the raw frame
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]

# Release the video stream and close the window
cv2.destroyAllWindows()


In [ ]:
#with sending to esp32
import cv2
import urllib.request
import numpy as np
import requests

# Define the ESP32 Wrover's IP address and port number
url = 'http://10.0.0.15:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            # Convert the frame to grayscale and detect faces
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            # Draw rectangles around the detected faces
            for (x, y, w, h) in faces:
                print("I see you!")
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                print("I see you!")

                # Send number 32 to ESP32 over WiFi
                requests.get('http://10.0.0.15/?data=32')
                response = requests.get(url + '/?data=32')
                print(response.text)
                print("Data send!")

            # Display the processed frame
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]

# Release the video stream and close the window
cv2.destroyAllWindows()


In [ ]:
import cv2
import urllib.request
import requests
import numpy as np
import socket
import time

# Load the face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://10.0.0.15:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500) 
        #buffer += stream.read(10000)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        #print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)

            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Define the ESP32's IP address and port number
                ip = "10.0.0.15"
                port = 80
    
                print("I see you!")

                # Define the integer to send
                message = 32

                # Create a socket object
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

                # Connect to the ESP32
                client_socket.connect((ip, port))

                # Send the integer as a string
                client_socket.send(str(message).encode())

                # Close the socket connection
                client_socket.close()
            # Display the output
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()


In [ ]:
import cv2
import urllib.request
import requests
import numpy as np
import socket

# Load the face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define the ESP32 Wrover's IP address and port number
url = 'http://10.0.0.15:81'

# Open the video stream from the ESP32 Wrover
stream = urllib.request.urlopen(url)

cv2.namedWindow('Video Stream', cv2.WINDOW_NORMAL)

# Loop through the frames in the video stream
while True:
    # Read the next frame from the video stream
    buffer = bytes()
    while True:
        buffer += stream.read(2500) 
        #buffer += stream.read(10000)
        a = buffer.find(b'\xff\xd8')
        b = buffer.find(b'\xff\xd9')
        #print(a, b)
        if a != -1 and b != -1:
            break
        elif a != -1 and b == -1:
            buffer = buffer[a:]
            break
    
    if a != -1 and b != -1 and b > a:
        #print(a,b)
        frame = cv2.imdecode(np.frombuffer(buffer[a:b+2], dtype=np.uint8), cv2.IMREAD_COLOR)

        if frame is not None:
            cv2.imshow('Video Stream', frame)

            # Apply the face detection model to the frame
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5)
            
            # Draw bounding boxes around the detected faces
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Define the ESP32's IP address and port number
                ip = "10.0.0.15"
                port = 81
    
                print("I see you!")

                # Define the integer to send
                message = 3

                # Create a socket object
                print("sending data")
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

                # Connect to the ESP32
                print("connecting so ESP32")
                client_socket.connect((ip, port))

                # Send the integer as a string
                print("sending message as a string")
                client_socket.send(str(message).encode())
                
                # Send number 32 to ESP32 over WiFi
                print("sending data again")
                requests.get('http://10.0.0.15/?data=32')
                
                print("Data send!")

                # Close the socket connection
                client_socket.close()
                
                
                
            # Display the output
            cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
        
    # Remove the processed data from the buffer
    buffer = buffer[b+2:]


# Release the video stream and close the window
cv2.destroyAllWindows()


I see you!
sending data
connecting so ESP32
sending message as a string
sending data again
Data send!
I see you!
sending data
connecting so ESP32
sending message as a string
sending data again
Data send!
I see you!
sending data
connecting so ESP32
sending message as a string
sending data again
Data send!
I see you!
sending data
connecting so ESP32
sending message as a string
sending data again
Data send!
